In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from paths_cpt import *

df = pd.read_parquet(PATH_TO_PARQUET)
print(df.shape)
df.head()

(1222711, 19)


,sondering_id,index,pkey_sondering,sondeernummer,x,y,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt,lithostrat_id
0,314,2593,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07,Quartair
1,314,2594,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07,Quartair
2,314,2595,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07,Quartair
3,314,2596,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09,Quartair
4,314,2597,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07,Quartair


In [16]:
df.lithostrat_id.unique()

array(['Quartair', 'Mont_Panisel', 'Aalbeke', None, 'Mons_en_Pevele',
       'Quartair + Mont_Panisel', 'Gentbrugge', 'Egem', 'Maldegem',
       'Brussel', 'Onbekend', 'Ursel', 'Asse', 'Wemmel', 'Lede',
       'Bolderberg', 'Tielt', 'Merelbeke', 'Kwatrecht', 'Antropogeen',
       'Vlierzele', 'Schelde Groep + Mons_en_Pevele',
       'Veldwezelt en Gembloux', 'Diest', 'Orchies', 'Tertiair',
       'Egemkapel', 'Sint_Huibrechts_Hern', 'Quartair_Holoceen',
       'Onbekend + Mont_Panisel'], dtype=object)

In [48]:
target_set = df[(df["lithostrat_id"].isna()) | (df["lithostrat_id"]=="None") | df["lithostrat_id"].str.contains('Onbekend')]
train_set = df[~((df["lithostrat_id"].isna()) | (df["lithostrat_id"]=="None") | df["lithostrat_id"].str.contains('Onbekend'))]
print("rows in train and target are : ", train_set.shape[0], " ; ", target_set.shape[0], ",",\
       target_set.shape[0] + train_set.shape[0], " = ", df.shape[0])

rows in train and target are :  196012  ;  1026699 , 1222711  =  1222711


In [49]:
target_set.lithostrat_id.unique()
train_set.lithostrat_id.unique()

array(['Quartair', 'Mont_Panisel', 'Aalbeke', 'Mons_en_Pevele',
       'Quartair + Mont_Panisel', 'Gentbrugge', 'Egem', 'Maldegem',
       'Brussel', 'Ursel', 'Asse', 'Wemmel', 'Lede', 'Bolderberg',
       'Tielt', 'Merelbeke', 'Kwatrecht', 'Antropogeen', 'Vlierzele',
       'Schelde Groep + Mons_en_Pevele', 'Veldwezelt en Gembloux',
       'Diest', 'Orchies', 'Tertiair', 'Egemkapel',
       'Sint_Huibrechts_Hern', 'Quartair_Holoceen'], dtype=object)

In [50]:
y = train_set.lithostrat_id
X = train_set.drop(columns = ["lithostrat_id"])
X.head()

,sondering_id,index,pkey_sondering,sondeernummer,x,y,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt
0,314,2593,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07
1,314,2594,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07
2,314,2595,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07
3,314,2596,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09
4,314,2597,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test.head())

(147009, 18) (49003, 18) (147009,) (49003,)
29839        Quartair
136132    Antropogeen
57074          Wemmel
376685        Brussel
103442          Diest
Name: lithostrat_id, dtype: object
